In [2]:
# Imports
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt


In [3]:
ensure_pyterrier_is_loaded()
tira = Client()

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [4]:
# The dataset: the union of the IR Anthology and the ACL Anthology
# This line creates an IRDSDataset object and registers it under the name provided as an argument.
pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

# A (pre-built) PyTerrier index loaded from TIRA
index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)

Download from the Incubator: https://files.webis.de/data-in-production/data-research/tira-zenodo-dump-preparation/ir-lab-sose2024/2024-05-04-16-05-53.zip
	This is only used for last spot checks before archival to Zenodo.


Download: 100%|██████████| 19.5M/19.5M [00:00<00:00, 45.5MiB/s]


Download finished. Extract...
Extraction finished:  /root/.tira/extracted_runs/ir-lab-sose-2024/ir-acl-anthology-20240504-training/tira-ir-starter


In [5]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

In [6]:

querys = pt_dataset.get_topics('query')
querys.head(10)

Download from the Incubator: https://files.webis.de/data-in-production/data-research/tira-zenodo-dump-preparation/ir-lab-sose2024/ir-acl-anthology-20240504-truth.zip?download=1
	This is only used for last spot checks before archival to Zenodo.


Download: 100%|██████████| 29.6k/29.6k [00:00<00:00, 1.61MiB/s]

Download finished. Extract...
Extraction finished:  /root/.tira/extracted_datasets/ir-lab-sose-2024/ir-acl-anthology-20240504-training/


,qid,query
0,1,retrieval system improving effectiveness
1,2,machine learning language identification
2,3,social media detect self harm
3,4,stemming for arabic languages
4,5,audio based animal recognition
5,6,comparison different retrieval models
6,7,cache architecture
7,8,document scoping formula
8,9,pseudo relevance feedback
9,10,how to represent natural conversations in word...


In [7]:
print(querys['query'][0])

retrieval system improving effectiveness


In [8]:
for i in range(10):
    print(querys['query'][i])

retrieval system improving effectiveness
machine learning language identification
social media detect self harm
stemming for arabic languages
audio based animal recognition
comparison different retrieval models
cache architecture
document scoping formula
pseudo relevance feedback
how to represent natural conversations in word nets


LEMMATIZATION

In [9]:
import nltk
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [10]:
def lemmatize(t):
    lemmatizer = WordNetLemmatizer()
    return lemmatizer.lemmatize(t)

In [11]:
testterm = "tests"
lemmatize(testterm)

'test'

In [32]:
query = querys['query'][67]
print(query)
tokens = query.split(' ')
print("Tokens: ",tokens)
lemma_query = list()
for token in tokens:
    lemma_query.append(lemmatize(token))

print("Lemmatized Tokens: ", lemma_query)

advancements in information retrieval
Tokens:  ['advancements', 'in', 'information', 'retrieval']
Lemmatized Tokens:  ['advancement', 'in', 'information', 'retrieval']


authors -> author

In [48]:
lemmq = list()
for query in querys['query']:
    tokens = query.split(" ")
    lemmatized_query = ""
    for token in tokens:
        lemmatized_query += " " + lemmatize(token)
    lemmq.append(lemmatized_query.strip())

print(lemmq)

['retrieval system improving effectiveness', 'machine learning language identification', 'social medium detect self harm', 'stemming for arabic language', 'audio based animal recognition', 'comparison different retrieval model', 'cache architecture', 'document scoping formula', 'pseudo relevance feedback', 'how to represent natural conversation in word net', 'algorithm acceleration with nvidia cuda', 'mention of algorithm', 'at least three author', 'german domain', 'mention of open source', 'inclusion of text mining', 'the ethic of artificial intelligence', 'machine learning for more relevant result', 'crawling website using machine learning', 'recommenders influence on user', 'search engine caching effect', 'consumer product review', 'limitation machine learning', 'medicine related research', 'natural language processing', 'graph based ranking', 'medical study that use information retrieval', 'information retrieval on different language source', 'paper that compare multiple informatio

In [55]:
for i in range(len(querys)):
    querys['query'][i] = lemmq[i]

querys.head(5)

,qid,query
0,1,retrieval system improving effectiveness
1,2,machine learning language identification
2,3,social medium detect self harm
3,4,stemming for arabic language
4,5,audio based animal recognition


In [66]:
run = bm25(querys)
run.head(5)

,qid,docid,docno,rank,score,query
0,1,94858,2004.cikm_conference-2004.47,0,15.681777,retrieval system improving effectiveness
1,1,125137,1989.ipm_journal-ir0volumeA25A4.2,1,15.047380,retrieval system improving effectiveness
2,1,125817,2005.ipm_journal-ir0volumeA41A5.11,2,14.144223,retrieval system improving effectiveness
3,1,5868,W05-0704,3,14.025748,retrieval system improving effectiveness
4,1,84876,2016.ntcir_conference-2016.90,4,13.947994,retrieval system improving effectiveness


In [67]:
persist_and_normalize_run(run, system_name='bm25-with-lemmatization', default_output='../runs')

The run file is normalized outside the TIRA sandbox, I will store it at "../runs".
Done. run file is stored under "../runs/run.txt".
